> Build a class to have all the schemas for a perticular graph problem
>> allow function like
>>> 1. generate ppl which represent the walks of some lengths 
>>> 2. mutate this populations // 
>>> 3. crossover this polulation // needs to be specific justifying why this preservs the best ones
>>> 4. select form this population // need to select until hit some number and then do selection 
>>> 5. at end we will have population which will represent set of optimal pats from which we can create new combinations of path

In [1]:
import numpy as np
#import networkx as nx
import itertools as itr
import time

In [286]:
class TSP_fullyconnected:
    
    
    # parameters on graphs on which TSP is done
    # none other parameters which are not of graph
    def graph(self,W):
        self.n=W.shape[0]
        self.W=W
        
        
    
    # function to get a schema of a desired length
    def get_schema(self,length):
        
        n=self.n
        d=np.arange(n)
        s=[]
        
        for i in range(length):
            
            k=np.random.randint(n)
            s.append(int(d[k]))
            d[k],d[n-1]=d[n-1],d[k]
            n-=1
            
        return s
    
    
    
    
    # returns the population of schema with fixed length and size
    def gen_ppl(self,size,length):
        
        ppl=[]
        limit = 100000
        i=size
        
        # warning to inform that limit number of schemas have been exceeded
        if size > limit:
            print("Warning : Limit Exceeded")
            
        while i > 0 and limit > 0:
            s=self.get_schema(length)
            if s not in ppl:
                ppl.append(s)
                i-=1
            limit-=1
            
        return ppl
    
    
    
    
    def fitness_funct(self,s):
        
        n=len(s)

        #fitness function
        w=0
        for i in range(n-1): 
            w+=1/(1+self.W[s[i]][s[i+1]])
        w/=n
        
        return w
    
    
    
    # default fitness function
    def eval_fitness(self,ppl,fitness=None):
        fit=[]
        for s in ppl:
            n=len(s)
            
            if fitness == None:
                #fitness function
                w=0
                for i in range(n-1): 
                    w+=1/(1+self.W[s[i]][s[i+1]])
                w/=n
            else:
                w = fitness(s)
                
            fit.append(w)
            
        return fit
        
    
    
    
    
    
    def mutate(self,ppl,m_rate):
        length=len(ppl[0])
        for s in range(len(ppl)):
                
            if np.random.rand() < m_rate:
            
                k = np.random.randint(length)
                r = list(set(np.arange(self.n))-set(ppl[s]))
                l = len(r)
                 
                if l > 0:
                    j = np.random.randint(l)
                    ppl[s][k]=r[j]
                
                
                
                
                
    # what proportion of them do crossover as c_rate
    def crossover(self,ppl,c_rate):
        length=len(ppl[0])
        fitness = np.array(self.eval_fitness(ppl))
        
        # pick random 2 from ppl
        # for roulet wheel crossover
        for i in range(len(fitness)-1):
            fitness[i+1]=fitness[i]+fitness[i+1]
        cdf_sum=int(fitness[len(ppl)-1]) 
        
        c_ppl=[]
        
        for times in range(len(ppl)):
            
            if np.random.rand() < c_rate:
                # index selected i and j
                k=cdf_sum*np.random.rand()
                i=np.searchsorted(fitness,k,side='right')-1
                k=cdf_sum*np.random.rand()
                j=np.searchsorted(fitness,k,side='right')-1

                # crossover them and generate 2 new samples add them to crossover set
                #crossover points
                p=np.random.randint(length)
                q=p+np.random.randint(length-p)
                """print('crossing ', ppl[i], ' and ' ,ppl[j],'with',p,'and',q)"""
                si=set(ppl[i][p:q+1])
                sj=set(ppl[j][p:q+1])

                ci=[]
                cj=[]
                for t in range(length-q+p-1):
                    ki=int(ppl[i][(t+q+1)%length])
                    kj=int(ppl[j][(t+q+1)%length])
                    if ki not in sj:
                        ci.append(ki)
                    if kj not in si:
                        cj.append(kj)

                for t in range(q-p+1):
                    ci.append(int(ppl[j][t+p]))
                    cj.append(int(ppl[i][t+p]))
                    
                
                #if length is less than length
                t=0
                while len(ci) < length:
                    if int(ppl[j][(t+q+1)%length]) not in ci:
                        ci.append(int(ppl[j][(t+q+1)%length]))
                    t+=1
                    
                t=0
                while len(cj) < length:
                    if int(ppl[i][(t+q+1)%length]) not in cj:
                        cj.append(int(ppl[i][(t+q+1)%length]))
                    t+=1
                    
                    
                #rotating lists 
                ci = ci[length-q-1:] + ci[:length-q-1]
                cj = cj[length-q-1:] + cj[:length-q-1]
                
                c_ppl.append(ci)
                c_ppl.append(cj)
                """print('result ', ci, ' and ' ,cj)
                print('_________________________')"""
                # repeat
        
    # return the crossover set
        return c_ppl
                
        
        
    def set_ppl(self,size):
        self.ppl_set=[]
        for i in range(self.n - 1):
            self.ppl_set.append(self.gen_ppl(size,i+2))
            
            
            
            
            
    def set_ppl_fitness(self,fitness=None):
        self.ppl_fitness_set=[]
        for i in range(self.n - 1):
            self.ppl_fitness_set.append(self.eval_fitness(self.ppl_set[i],fitness=fitness))
    
    
    
    
    
    def sort_ppl(self,ppl,fit):
        return [ppl for fit, ppl in sorted(zip(fit, ppl))]
            
        
        
        
    def sort_select(self, ppl , fit ,s_rate=0.5):
        N=int(s_rate*len(ppl))
        selected_ppl = self.sort_ppl(ppl,fit)
        return selected_ppl[-N:]
    
    
    
    
    
    def ppl_sort_select(self,s_rate=0.5):
        for i in range(len(self.ppl_set)):
            self.ppl_set[i] = self.sort_select(self.ppl_set[i],self.ppl_fitness_set[i],s_rate=s_rate)
            
            
            
            
            
    def ppl_crossover_set(self,c_rate):
        self.crossover_set=[]
        for i in range(len(self.ppl_set)):
            self.crossover_set.append(self.crossover(self.ppl_set[i],c_rate=c_rate))
            
            
            
            
    def ppl_mutate(self,m_rate):
        for i in range(len(self.ppl_set)):
            self.mutate(self.ppl_set[i],m_rate=m_rate)
            
            
            
            
    def run(self,size,epoch,c_rate=0.5,s_rate=0.7,m_rate=0.2,fitness=None):
        
        self.set_ppl(size)
        while epoch > 0:
            self.ppl_crossover_set(c_rate)
            for i in range(len(self.ppl_set)):
                self.ppl_set[i]+=self.crossover_set[i]
            self.ppl_mutate(m_rate)
            self.set_ppl_fitness(fitness=fitness)
            self.ppl_sort_select(s_rate)
            epoch-=1
            #print(epoch)
        self.set_ppl_fitness(fitness=fitness)

In [287]:
t=TSP_fullyconnected()

In [288]:
t.graph(np.random.rand(10,10))

In [289]:
t.run(20,20,0.5,0.5,0.2)

In [290]:
t.W

array([[0.47342745, 0.81661269, 0.14782578, 0.28932129, 0.69745991,
        0.13805594, 0.50110483, 0.55222498, 0.15817598, 0.42100591],
       [0.40429363, 0.55503767, 0.24990153, 0.71443121, 0.51349631,
        0.72875756, 0.52199426, 0.3212214 , 0.06613057, 0.07162554],
       [0.26000906, 0.6842484 , 0.21502234, 0.60822058, 0.8473643 ,
        0.17758873, 0.23245007, 0.60691376, 0.65480433, 0.86333854],
       [0.95095242, 0.78040898, 0.31637479, 0.54150985, 0.43567385,
        0.40250476, 0.66065487, 0.52922398, 0.39481225, 0.10911927],
       [0.51260112, 0.47234751, 0.60397957, 0.1363313 , 0.26467887,
        0.1974533 , 0.58738586, 0.46998053, 0.65884572, 0.48905699],
       [0.33698805, 0.29422878, 0.66457978, 0.31304949, 0.51447102,
        0.32662591, 0.59054141, 0.61755094, 0.47082974, 0.832878  ],
       [0.35640622, 0.54447769, 0.01763574, 0.56927731, 0.6984324 ,
        0.62507068, 0.09201729, 0.45750051, 0.00496212, 0.29869569],
       [0.3757064 , 0.67248409, 0.0256575

below we can see final results

In [293]:
(t.ppl_set[0],t.ppl_fitness_set[0])

([[6, 8], [6, 8], [6, 8], [6, 8], [6, 8], [6, 8], [6, 8]],
 [0.49753118857044987,
  0.49753118857044987,
  0.49753118857044987,
  0.49753118857044987,
  0.49753118857044987,
  0.49753118857044987,
  0.49753118857044987])

In [184]:
ppl=t.gen_ppl(10,5)
ppl

[[9, 8, 7, 3, 2],
 [7, 5, 0, 4, 9],
 [5, 8, 2, 6, 7],
 [3, 0, 9, 8, 7],
 [7, 0, 3, 9, 8],
 [8, 1, 7, 4, 6],
 [2, 5, 0, 3, 6],
 [3, 2, 1, 7, 8],
 [4, 9, 2, 6, 0],
 [4, 3, 7, 0, 1]]

In [185]:
t.crossover(ppl,0.5)

[[7, 5, 0, 3, 6],
 [2, 5, 0, 4, 9],
 [4, 3, 8, 7, 0],
 [3, 9, 0, 1, 4],
 [9, 8, 7, 3, 6],
 [5, 0, 3, 2, 9],
 [7, 1, 0, 4, 9],
 [8, 5, 7, 4, 6]]

In [186]:
t.mutate(ppl,0.5)
ppl

[[9, 8, 7, 3, 2],
 [7, 5, 0, 4, 9],
 [5, 8, 2, 6, 7],
 [1, 0, 9, 8, 7],
 [7, 0, 3, 9, 4],
 [8, 1, 7, 4, 6],
 [2, 5, 0, 3, 6],
 [4, 2, 1, 7, 8],
 [4, 9, 2, 6, 0],
 [4, 3, 7, 2, 1]]

In [240]:
epoch

20

In [245]:
t.set_ppl(20)

In [238]:
t.ppl_set

[[[3, 1],
  [6, 9],
  [0, 9],
  [3, 2],
  [7, 3],
  [0, 3],
  [6, 5],
  [5, 9],
  [6, 3],
  [9, 3],
  [9, 0],
  [3, 6],
  [7, 9],
  [8, 3],
  [0, 1],
  [2, 0],
  [3, 8],
  [6, 7],
  [1, 8],
  [0, 2]],
 [[1, 8, 3],
  [8, 0, 5],
  [4, 5, 8],
  [6, 5, 3],
  [4, 9, 7],
  [6, 8, 4],
  [5, 2, 7],
  [8, 3, 7],
  [5, 0, 4],
  [1, 5, 9],
  [7, 4, 3],
  [7, 8, 9],
  [2, 1, 6],
  [9, 8, 5],
  [3, 4, 8],
  [4, 6, 8],
  [4, 3, 2],
  [1, 8, 6],
  [6, 0, 2],
  [1, 3, 8]],
 [[4, 1, 2, 7],
  [2, 1, 4, 8],
  [1, 5, 0, 6],
  [2, 7, 8, 5],
  [4, 0, 5, 7],
  [9, 8, 5, 3],
  [1, 2, 7, 3],
  [2, 5, 0, 9],
  [8, 6, 3, 7],
  [6, 1, 8, 2],
  [2, 0, 3, 8],
  [2, 9, 7, 6],
  [7, 3, 6, 8],
  [5, 7, 3, 6],
  [6, 5, 0, 3],
  [8, 0, 4, 5],
  [6, 4, 8, 9],
  [4, 1, 8, 3],
  [1, 2, 0, 7],
  [1, 2, 6, 4]],
 [[5, 4, 1, 3, 2],
  [2, 1, 9, 6, 0],
  [3, 9, 1, 7, 2],
  [9, 8, 0, 5, 4],
  [1, 5, 3, 2, 0],
  [4, 9, 1, 3, 8],
  [1, 8, 4, 3, 7],
  [1, 0, 3, 8, 9],
  [8, 0, 3, 5, 2],
  [1, 0, 9, 2, 8],
  [2, 7, 1, 9, 6],
  [6, 9,

In [236]:
t.ppl_set[2]

In [189]:
t.eval_fitness(t.ppl_set[8])

[0.6245264187725623,
 0.6424908369930832,
 0.6384919235951028,
 0.596244822508602,
 0.7079346030634011]

In [190]:
t.set_ppl_fitness()

In [162]:
t.ppl_set[7]

[[7, 8, 1, 9, 0, 6, 4, 2, 5],
 [1, 7, 2, 8, 0, 3, 5, 9, 6],
 [2, 5, 9, 0, 7, 3, 8, 6, 4],
 [4, 8, 2, 0, 7, 3, 1, 6, 9],
 [1, 3, 8, 6, 2, 5, 4, 7, 0]]

In [163]:
t.ppl_fitness_set[7]

[0.6662532121124516,
 0.7059503819510584,
 0.6707400160910123,
 0.5889032618001333,
 0.6565248684061648]

In [164]:
t.sort_select( t.ppl_set[7] , t.ppl_fitness_set[7] )

[[2, 5, 9, 0, 7, 3, 8, 6, 4], [1, 7, 2, 8, 0, 3, 5, 9, 6]]

In [191]:
t.ppl_crossover_set(0.5)

[[7, 9, 4, 6, 0, 1, 3],
 [5, 3, 0, 4, 9, 8, 7],
 [7, 1, 4, 2, 9, 8, 3],
 [1, 3, 7, 4, 6, 0, 2],
 [7, 4, 3, 6, 1, 5, 0],
 [7, 5, 4, 2, 8, 6, 0]]

In [317]:
list((set(np.arange(9))-set(np.arange(9))))

[]

In [189]:
class schemas:
    
    def convert_to_adj_list(self):
        A=self.A
        n=self.graph_size
        W= self.weight
        adj={}
        for i in range(n):
            adj[i]={}
            for j in range(n):
                if A[i,j] : adj[i][j] = W[i,j]
        self.adj=adj
                    
            
    def graph(self,A,W=None):
        self.A=A
        self.graph_size=A.shape[0]
        if W is None: W=np.zeros(A.shape)+1
        self.weight=W
        
        
    def specify(self,count,max_length):
        self.count=count
        self.max_length = max_length
        
    def set_ppl_size(self,ppl_size):
        self.ppl_size=ppl_size
        
    def set_target_source(self,t,s):
        self.target = t
        self.source = s
    
        
    def generate_ppl(self):
        
        self.ppl=[]
        def get_schema():
            # random number as length of schema max
            m=int(1+np.random.randint(self.max_length,size=1))
            adj=self.adj
            s=[]
            
            # starting point of schema
            k=int(np.random.randint(self.graph_size,size=1))
            while len(list(adj[k].keys())) is 0: k=int(np.random.randint(self.graph_size,size=1))
            # while there is way to from to other from the starting point 
            
            s=[k]
            
            last=k
            for i in range(m):
                
                #generate new node to go which is adjecent to the new currently on
                j=int(list(adj[k].keys())[int(np.random.randint(len(list(adj[k].keys())),size=1))])
                while j is last:j=int(list(adj[k].keys())[int(np.random.randint(len(list(adj[k].keys())),size=1))])
                # do untill this new node is different from the last visited node
                s.append(j)
                
                #break if reached pendent node
                if len(list(adj[j].keys())) < 2 :break
                
                last=k
                k=j
            #print(s)
            return s
        
        for i in range(self.ppl_size):
            temp=get_schema()
            self.ppl.append(temp)
            
    def remove_duplicate(self):
        x=self.ppl
        y=[]
        for i in x:
            if i not in y:
                y.append(i)

        self.ppl = y
            
    def eval_fitness(self):
        fit=[]
        for s in self.ppl:
            n=len(s)
            
            #fitness function
            w=0
            for i in range(n-1): 
                w+=1/(1+F.weight[s[i]][s[i+1]])
            w/=n
            fit.append(w)
        self.fitness = fit
        
        
    def mutate(self,m_rate):
        
        for s in range(len(self.ppl)):
            if np.random.rand() < m_rate:
                
                l = len(self.ppl[s])
                k = np.random.randint(l)
                
                if k is 0:
                    p = np.random.randint(sum(self.A[1]))
                    i=0
                    while p > 0 and i < self.graph_size:
                        if self.A[1][i] is 1:
                            p-=1
                        i+=1
                    self.ppl[s][0]=i
                        
                if k is l-1:
                    p = np.random.randint(sum(self.A[l-2]))
                    i=0
                    while p > 0 and i < self.graph_size:
                        if self.A[l-2][i] is 1:
                            p-=1
                        i+=1
                    self.ppl[s][l-1]=i
                
                if k > 0 and k < l-1:
                    D = self.A[k-1]*self.A[k+1]
                    p = np.random.randint(sum(D))
                    i=0
                    while p > 0 and i < self.graph_size:
                        if D[i] is 1:
                            p-=1
                        i+=1
                    self.ppl[s][k]=i
                    

In [190]:
np.random.randint(1)

0

In [191]:
F.A[0]*F.A[2]

array([0, 1, 0, 1, 1])

In [192]:
F=schemas()

In [193]:
F.specify(20,4)

In [194]:
F.graph(A=np.array([[0,1,1,1,1],[1,0,1,1,1],[1,1,0,1,1],[1,1,1,0,1],[1,1,1,1,0]]),W=np.array([[0,5,3,2,3],[4,7,0,0,5],[7,8,0,0,4],[4,8,8,5,9],[7,8,0,0,4]]))

In [195]:
F.convert_to_adj_list()

In [196]:
list(F.adj[3].keys())

[0, 1, 2, 4]

In [197]:
F.set_ppl_size(100)

In [198]:
F.set_target_source(3,2)
F.target
#F.source

3

In [205]:
F.generate_ppl()
F.remove_duplicate()
len(F.ppl)

90

In [207]:
F.mutate(0.1)

In [432]:
#import inspect

In [429]:
#inspect.ismethod(np.random.rand)

In [430]:
#inspect.getmembers(inspect)

In [431]:
#dir(inspect)